# Ch17.Future를 이용한 동시성
concurrent.furures 라이브러리를 중점적으로...
비동기 작업의 실행을 나타내는 객체인 Future의 개념에 대해 소개한다.
이 강력한 개념은 후에 asyncio 패키지의 기반이 된다. asyncio는 18장에서...

## 17.1 예제: 세 가지 스타일의 웹 내려받기
긴 지연 시간동안 CPU 클록을 낭비하지 않기 우해 네트워크 입출력을 효육적으로 처리하ㅁ려면 동시성을 이용해야 한다. 네트워크에서 응답이 오는 동안 다른 일을 처리하는 것이 좋다.  
동시성을 활용하는 방법을 코드로 보여주기 위해서 웹에서 20개 국가의 국기 이미지를 내려받는 간단한 프로그램을 3개 작성한다. 첫 번째 flags.py는 순차적으로 실행되므로 이전 이미지를 내려받아 디스크에 저장한 후 다음번 이미지를 내려받는다. 나머지 프로그램 두 개는 동시에 내려받는다. 즉, **모든 이미지를 동시에 요청한 후 도착하는 대로** 파일에 저장한다.

### 17.1.1 순차 내려받기 스크립트
`flags.py`는 그리 흥미롭지 않지만, 이 코드 대부분을 동시성 스크립트를 구현하는 데 다시 사용하므로, 주의해서 봐둘 필요가 있다

In [9]:
!python flags.py

BD BR CD CN ED EG ET ID IN IR JP KR MX NG PH PK RU TR US VN 
20 flags downloaded in 15.78s


In [12]:
!python flags.py

BD BR CD CN ED EG ET ID IN IR JP KR MX NG PH PK RU TR US VN 
20 flags downloaded in 8.79s


### 17.1.2 concurrent.futures로 내려받기
`concurrent.futures` 패키지의 가장 큰 특징은 ThreadPoolExecutor와 ProcessPoolExecutor클래스인데, 이 클래스들은 콜러블 객체를 서로 다른 스레드나 프로세스에서 실행할 수 있게 해주는 인터페이스를 구현한다. 이 클래스들은 작업자 스레드나 작업자 프로세스를 관리하는 풀과 실행할 작업을 담은 큐를 가지고 있다. 그러나 아주 고수준의 인터페이스를 구현하고 있어서 국기를 내려받는 간단한 프로그램을 구현할 때는 내부의 작동 과정을 알 필요가 없다.

In [7]:
!python flags_threadpool.py

ID BD RU IN MX EG BR PH US TR VN IR NG PK KR CN ET JP ED CD 
20 flags downloaded in 1.40s


In [17]:
!python flags_threadpool.py

CN KR BD JP US ID MX IR TR NG PH ET VN CD IN RU BR EGPK  ED 
20 flags downloaded in 0.95s


download_one()함수는 download_many()의 for 루프 본체와 본질적으로 동일하다. 순차적으로 실행되는 for 루프 본체를 동시에 호출할 함수로 바꾼것이다.
라이브러리 이름이 concurrency.futures인데, 여기서는 future는를 볼 수 없다. Future가 어딨어

### 17.1.3. Future는 어디에 있나?
Future는 concurrent.futures와 asyncio의 내부에 있는 핵심 컴포넌트인데, 이 라이브러리의 사용자에게 드러나지 않는 경우가 종종 있다.
파이썬 3.4 표준 라이브러리에서 Future는 `concurrent.futures`와 `asyncio.Future`가 있다. 이 두 클래스는 완료되었을 수도 있고 아닐 수도 있는 지연된 계산을 표현하기 위해 사용된다. Future 클래스는 Twisted의 Deferred 클래스, Tornado의 Future클래스, 자바스크립트 라이브러리의 Promise 객체와 비슷하다.

In [18]:
!python flags_threadpool_ac.py

Scheduled forBR: <Future at 0x105d3fb80 state=running>
Scheduled forCN: <Future at 0x105d86ca0 state=running>
Scheduled forID: <Future at 0x105d86f10 state=running>
Scheduled forIN: <Future at 0x105d86b80 state=pending>
Scheduled forUS: <Future at 0x105da10d0 state=pending>
ID <Future at 0x105d86f10 state=finished returned str> result: ID
CN BR<Future at 0x105d86ca0 state=finished returned str> result: CN
 <Future at 0x105d3fb80 state=finished returned str> result: BR
IN <Future at 0x105d86b80 state=finished returned str> result: IN
US <Future at 0x105da10d0 state=finished returned str> result: US

5 flags downloaded in 0.91s


In [20]:
!python flags_threadpool_ac.py

Scheduled forBR: <Future at 0x10244bb80 state=running>
Scheduled forCN: <Future at 0x102491ca0 state=running>
Scheduled forID: <Future at 0x102491f10 state=running>
Scheduled forIN: <Future at 0x1024a88b0 state=pending>
Scheduled forUS: <Future at 0x1024ae2b0 state=pending>
ID BR <Future at 0x10244bb80 state=finished returned str> result: BR
<Future at 0x102491f10 state=finished returned str> result: ID
CN <Future at 0x102491ca0 state=finished returned str> result: CN
IN US <Future at 0x1024a88b0 state=finished returned str> result: IN
<Future at 0x1024ae2b0 state=finished returned str> result: US

5 flags downloaded in 0.92s


## 17.2 블로킹 I/O와 GIL(Global Interpreter Lock)
Cpython 인터프리터는 내부적으로 스레드 안전하지 않으므로, 전역 인터프리터 락(GIL)을 가지고 있다. GIL은 한 번에 한 스레드만 파이썬 바이트코드를 실행하도록 제한한다. 그렇기 때문에 단일 파이썬 프로세스가 동시에 다중 CPU 코어를 사용할 수 없다.  
파이썬 코드를 작성할 때 우리는 GIL을 제어할 수 없지만, 내장 함수나 C로 작성된 확장은 시간이 오래 걸리는 작업을 실행할 때 GIL을 해제할 수 있다. 사실 C로 작성된 파이썬 라이브러리는 GIL을 관리하고, 자신의 OS 스레드를 생성해서 가용한 CPU 코드를 모두 사용할 수 있다 하지만 라이브러리 코드가 상당히 복잡해지므로, 대부분의 라이브러리 제작자는 이런 방식으로 구현하지 않는다.  
그런데 블로킹 입출력을 실행하는 모든 표준 라이브러리 함수는 OS에서 결과를 기다리는 동안 GIL을 해제한다. 즉, 입출력 위주의 작업을 실행하는 파이썬 프로그램은 파이썬으로 구현하더라도 스레드를 이용함으로써 이득을 볼 수 있다는 것이다. 아하!

concurrent.futures를 사용해서 계산 위주의 작업에서 GIL을 우회하는 간단한 방법을 살펴보자.

## 17.3 concurrent.futures로 프로세스 실행하기
ProcessPoolExecutor는 GIL을 위회하므로 계산 위주의 작업을 수행해야하는 경우 가용한 CPU를 모두 사용한다.
ProcessPoolExecutor와 ThreadPoolExecutor는 모두 범용 Executor 인터페이스를 구현하므로, concurrent.futures를 사용하는 경우에는 스레드 기반의 프로그램을 프로세스 기반의 프로그램으로 쉽게 변환할 수 있다.  
국기를 내려받는 프로그램처럼 입출력 위주의 작업에서는 ProcessPoolExecutor를 사용해도 도움이 안 된다. 



In [3]:
!python flags_threadpool.py     #ProcessPool 이용

EG ID CD BD BR ET CN ED JP NG IN KR IR MX PK PH US TR RU VN 
20 flags downloaded in 2.07s


In [4]:
!python flags_threadpool.py     # Threadpool 이용

NG MX US ET CD JP TR ID PH PK CN EDKR  IR BR BDIN  VN RU EG 
20 flags downloaded in 1.11s


간단히 사용할 때는 max_workers 인수를 통해 풀에 스레드 수를 지정해야 하는 차이점만 보인다. `ProcessPool`에서는 `os.cpu_count()`가 반환하는 값을 사용하므로 대부분 직접 설정하지 않는다. 
계산 위주의 작업을 할 때는 CPU보다 많은 작업수를 요청하는 것은 의미가 없으므로 별도로 지정하지 않는다. 하지만 입출력 위주의 연산을 수행하는 경우에는 ThreadPool에 수십 내지 수천 개의 스레드를 사용할 수 있다. 최적의 스레드 수는 **처리할 작업의 특성과 가용한 메모리에 따라 다르다**

## 17.4 Executor.map() 실험
`flags.py`에서 본것 처럼 `Executor.map()`을 이용하면 여러 콜러블을 아주 간단히 동시에 실행할 수 있다. 아래 코드는 Executor.map()이 작동하는 과정을 자세히 보여준다.

In [12]:
from time import sleep, strftime
from concurrent import futures

def display(*args):
    """자신이 받은 인수 앞에 [HH:MM:SS]포맷의 타임스탬프를 찍어서 출력한다."""
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)

def loiter(n):
    """시작할 때 메세지를 출력하고, 인수로 받은 n초 동안 잠자고, 마지막 메시지를 출력한다. 
    메시지 앞에 n개의 탭을 붙여 메세지를 들여 쓴다.
    """
    msg = '{}loiter({}): doing nothing for {}s...'
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = '{}loiter({}): done.'
    display(msg.format('\t'*n, n, n))
    return n * 10

def main():
    display('Script starting...')
    excutor = futures.ThreadPoolExecutor(max_workers=3)
    results = excutor.map(loiter, range(5))
    display('results:', results)
    display('Waiting for idividual results:')
    for i, result in enumerate(results):
        display('result {}: {}'.format(i, result))

In [13]:
main()

[18:51:38] Script starting...
[18:51:38] loiter(0): doing nothing for 0s...
[18:51:38] loiter(0): done.
[18:51:38] 	loiter(1): doing nothing for 1s...
[18:51:38] 		loiter(2): doing nothing for 2s...
[18:51:38] results: <generator object Executor.map.<locals>.result_iterator at 0x104af19e0>
[18:51:38] Waiting for idividual results:
[18:51:38] result 0: 0
[18:51:38] 			loiter(3): doing nothing for 3s...
[18:51:39] 	loiter(1): done.
[18:51:39] 				loiter(4): doing nothing for 4s...
[18:51:39] result 1: 10
[18:51:40] 		loiter(2): done.
[18:51:40] result 2: 20
[18:51:41] 			loiter(3): done.
[18:51:41] result 3: 30
[18:51:43] 				loiter(4): done.
[18:51:43] result 4: 40


Executor.map()은 사용하기 쉽지만, 호출한 순서 그대로 결과를 반환하는 특징이 있다. 이러한 특징은 상황에 따라 도움이 되기도 하고 아닐 수도 있다. 첫 번째 호출이 결과를 생성할 때까지 10초 걸리고 나머지 호출은 1초씩 걸린다면, map()이 반환한 제너레이터가 첫 번째 결과를 가져오기까지 10초 걸린다. 그 후 다른 함수는 이미 실행을 완료했을 테니 나머지 결과는 바로 가져올 수 있다. 더 진행하기 전에 모든 결과가 필요한 경우라면 이 특징은 문제가 되지 않지만, submit()한 순서와 상관없이 완료되는 대로 결과를 가져오는 게 더 좋은 경우도 있다.
완료되는 대로 결과를 가져오려면 excutor.submit()메서드와 futures.as_completed()함수를 함께 사용해야 한다. 이것은 17.5.2 절에서 설명한다.

## 17.5 진행 상황 출력하고 에러를 처리하며 내려받기

In [16]:
import time
from tqdm import tqdm
for i in tqdm(range(100)):
    time.sleep(.01)

100%|██████████| 100/100 [00:01<00:00, 80.22it/s]


tqdm()은 len()을 지원하는 반복형을 받거나, 예상 항목 수를 두 번째 인수로 받는다.

In [23]:
!python flags2_sequential.py -s DELAY a b c # 외않되...

DELAY site: http://localhost:8002/flags
Searching for 78 flags: from AA to CZ
1 concurrent connection will be used.
100%|██████████████████████████████████████████| 78/78 [00:00<00:00, 965.04it/s]
--------------------
0 flags downloaded.
78 errors.
Elapsed time: 0.09s
